In [1]:
from IPython.display import clear_output

import pandas as pd
import numpy as np

import requests


In [56]:
def get_json(endpoint,params = {},clint_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    r = requests.get(endpoint, params, auth=(clint_id,''))
    json = r.json()
    if r.status_code == 200:
        data = json['data']
        print('Data retrieved from frost.met.no')
    else:
        print('Error! Returned status code %s' % r.status_code, '; Message: %s' % json['error']['message'], '; Reason: %s' % json['error']['reason'])
        return -1
    return(data)


def get_df(endpoint,params = {},client_id = '23b60ca7-860f-4c84-bb2a-431206a0681d'):
    data = get_json(endpoint,params,client_id)
    df = pd.DataFrame()
    for i in range(len(data)):
        row = pd.DataFrame(data[i]['observations'])
        row['referenceTime'] = data[i]['referenceTime']
        row['sourceId'] = data[i]['sourceId']
        df = df.append(row)

    return(df.reset_index())
    

def get_df_row(json):
    
    df_row = pd.io.json.json_normalize(json)
    
    return(df_row)
    

In [29]:
location_df = pd.read_csv('../csv/locations.csv')

In [101]:
params = {
    'elements': 'mean(air_temperature P1D),
    'referencetime': '2012-01-01/2018-05-01',
}

SyntaxError: EOL while scanning string literal (<ipython-input-101-968d782790da>, line 2)

In [102]:
df_list = []
error_list = []
success_list = []

for i,loc in location_df.iloc[:100].iterrows():
    loc_params = params.copy()
    loc_params['sources'] = loc.station_id
    
    loc_json = get_json('https://frost.met.no/observations/v0.jsonld',loc_params)
    if loc_json != -1:
        df_list.append(get_df_row(loc_json[0]['observations']))
        success_list.append((loc['name'],loc['station_id']))
    else:
        error_list.append((loc['name'],loc['station_id']))

Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Data retrieved from frost.met.no
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Data retrieved from frost.met.no
Data retrieved from frost.met.no
Data retrieved from frost.met.no
Data retrieved from frost.met.no
Data retrieved from frost.met.no
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Data retrieved from frost.met.no
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Data retrieved from frost.met.no
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Data retrieved from frost.met.no
Data retrieved from frost.met.no
Error! Returned status code 404 ; Message: Not found ; Reason: No data found
Data retrieved from frost.met.no
Error! Returned statu

KeyboardInterrupt: 

In [91]:
ids = [x[1] for x in error_list]

In [103]:
error_list

[('Adamstuen', 'SN18390'),
 ('Aker brygge', 'SN18670'),
 ('Alsvåg', 'SN86950'),
 ('Alta', 'SN93090'),
 ('Ambjørnrød', 'SN3050'),
 ('Ammerud', 'SN18245'),
 ('Andenes', 'SN87105'),
 ('Ankenes', 'SN84701'),
 ('Apenesfjellet', 'SN3019'),
 ('Arendal', 'SN36110')]

In [93]:
len(np.unique(ids))

32

In [96]:
success_list

[('Adamstuen', 'SN18390'),
 ('Ajer', 'SN12320'),
 ('Akkerhaugen', 'SN32060'),
 ('Aksdal', 'SN47060'),
 ('Algrøyna', 'SN50514')]